
## Task 1: Binary Representations

In [ ]:
import numpy as np

## Task 2: Hash Functions

## Task 3: SHA256

## Task 4: Prime Numbers

## Task 5: Roots

## Task 6: Proof of Work

## Task 7: Turing Machines

## Task 8: Computational Complexity